In [2]:
import pandas as pd
import numpy as np
import geopandas as gpp
import seaborn as sns

df_votacao_candidato_munzona = pd.read_csv('/Users/willianribeiro/Documents/GitHub/analise_exploratoria_dados_tse/data/Resultados/2020/votacao_candidato_munzona_2020/votacao_candidato_munzona_2020_BRASIL.csv', sep=';', encoding='Latin 1')
df_votacao_partido_munzona = pd.read_csv('/Users/willianribeiro/Documents/GitHub/analise_exploratoria_dados_tse/data/Resultados/2020/votacao_partido_munzona_2020/votacao_partido_munzona_2020_BRASIL.csv', sep=';', encoding='Latin 1')
df_detalhe_votacao_munzona = pd.read_csv('/Users/willianribeiro/Documents/GitHub/analise_exploratoria_dados_tse/data/Resultados/2020/detalhe_votacao_munzona_2020/detalhe_votacao_munzona_2020_BRASIL.csv', sep=';', encoding='Latin 1')


In [3]:
df_votacao_candidato_munzona.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'NR_FEDERACAO', 'NM_FEDERACAO', 'SG_FEDERACAO',
       'DS_COMPOSICAO_FEDERACAO', 'SQ_COLIGACAO', 'NM_COLIGACAO',
       'DS_COMPOSICAO_COLIGACAO', 'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_NOMINAIS',
       'NM_TIPO_DESTINACAO_VOTOS', 'QT_VOTOS_NOMINAIS_VALIDOS',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO'],
      dtype='object')

In [4]:
df_votacao_partido_munzona.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'TP_AGREMIACAO',
       'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'NR_FEDERACAO',
       'NM_FEDERACAO', 'SG_FEDERACAO', 'DS_COMPOSICAO_FEDERACAO',
       'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
       'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_LEGENDA_VALIDOS',
       'QT_VOTOS_NOMINAIS_CONVR_LEG', 'QT_TOTAL_VOTOS_LEG_VALIDOS',
       'QT_VOTOS_NOMINAIS_VALIDOS', 'QT_VOTOS_LEGENDA_ANUL_SUBJUD',
       'QT_VOTOS_NOMINAIS_ANUL_SUBJUD'],
      dtype='object')

In [5]:
df_detalhe_votacao_munzona.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'QT_APTOS',
       'QT_SECOES_PRINCIPAIS', 'QT_SECOES_AGREGADAS',
       'QT_SECOES_NAO_INSTALADAS', 'QT_TOTAL_SECOES', 'QT_COMPARECIMENTO',
       'QT_ELEITORES_SECOES_NAO_INSTALADAS', 'QT_ABSTENCOES',
       'ST_VOTO_EM_TRANSITO', 'QT_VOTOS', 'QT_VOTOS_CONCORRENTES',
       'QT_VOTOS_VALIDOS', 'QT_VOTOS_NOMINAIS_VALIDOS',
       'QT_TOTAL_VOTOS_LEG_VALIDOS', 'QT_VOTOS_LEGENDA_VALIDOS',
       'QT_VOTOS_NOMINAIS_CONVR_LEG', 'QT_TOTAL_VOTOS_ANULADOS',
       'QT_VOTOS_NOMINAIS_ANULADOS', 'QT_VOTOS_LEGENDA_ANULADOS',
       'QT_TOTAL_VOTOS_ANUL_SUBJUD', 'QT_VOTOS_NOMINAIS_ANUL_SUBJUD',
       'QT_VOTOS_LEGENDA_ANUL_SUBJUD', 'QT_VOTOS_BRANCOS',
       'QT_TOTAL_VOTOS_NULOS', 'QT_VOTOS_NULOS', 'QT_VOTOS_NULO_TECNICO',


# Sobre as métricas que queremos fornecer:
- Quociente Eleitoral
- Quartil Partidário
- Total de Eleitores por Municipio
- Total de abstenções por Municipio
- Total de votos em branco
- Gráfico com total de votos por candidato
- Gráfico total de votos por partido
- Total de Candidatos Eleitos
- Clausula de Barreira da Eleição

### Quociente Eleitoral 

Definido como Total de Votos validos dividido Pela quantidade de cadeiras disponíveis

In [6]:
def contador_eleitos(dataset_tratado_cidade_cargo):
    dicionario_eleicao =  {
        'SUPLENTE': 0,
        'NÃO ELEITO': 0,
        'ELEITO POR QP': 0,
        'ELEITO POR MÉDIA': 0
    }

    # Conjunto para armazenar nomes de candidatos já encontrados
    nomes_encontrados = set()

    # Iterar pelas linhas do DataFrame
    for index, row in dataset_tratado_cidade_cargo.iterrows():
        # Verificar se o nome do candidato já apareceu
        if row['NM_CANDIDATO'] in nomes_encontrados:
        # Ignora a linha se o nome do candidato ja apareceu
            pass
        else:
            nomes_encontrados.add(row['NM_CANDIDATO'])
            dicionario_eleicao[row['DS_SIT_TOT_TURNO']] += 1

    return dicionario_eleicao

In [7]:
cidade = 'SÃO PAULO'

cargo = 'Vereador'

df_cidade_filtrado = df_votacao_candidato_munzona[df_votacao_candidato_munzona['NM_MUNICIPIO'] == cidade]

df_cidade_filtrado_cargo = df_cidade_filtrado[df_cidade_filtrado['DS_CARGO'] == cargo]

resultados_eleitos = contador_eleitos(df_cidade_filtrado_cargo)

cadeiras = resultados_eleitos['ELEITO POR QP'] + resultados_eleitos['ELEITO POR MÉDIA']

total_votos = df_cidade_filtrado_cargo['QT_VOTOS_NOMINAIS_VALIDOS'].sum()

quociente = total_votos / cadeiras

print(f'Total de cadeiras: {cadeiras}')
print(f'Total de votos: {total_votos}')
print(f'Quociente eleitoral: {quociente}')

Total de cadeiras: 55
Total de votos: 4427182
Quociente eleitoral: 80494.21818181819


In [21]:
df_cidade_filtrado_cargo.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'NR_FEDERACAO', 'NM_FEDERACAO', 'SG_FEDERACAO',
       'DS_COMPOSICAO_FEDERACAO', 'SQ_COLIGACAO', 'NM_COLIGACAO',
       'DS_COMPOSICAO_COLIGACAO', 'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_NOMINAIS',
       'NM_TIPO_DESTINACAO_VOTOS', 'QT_VOTOS_NOMINAIS_VALIDOS',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO'],
      dtype='object')

### Quociente Partidário

Definido como Total de Votos validos dividido pela quantidade de partidos

In [22]:
def calculadora_partido(dataset_tratado_cidade_cargo):
    dicionario_partido = {}

    # Conjunto para armazenar nomes de candidatos já encontrados
    partidos_encontrados = set()

    # Iterar pelas linhas do DataFrame
    for index, row in dataset_tratado_cidade_cargo.iterrows():
        # Verificar se o nome do candidato já apareceu
        if row['SG_PARTIDO'] in partidos_encontrados:
            dicionario_partido[row['SG_PARTIDO']] += row['QT_VOTOS_NOMINAIS_VALIDOS']
        else:
            partidos_encontrados.add(row['SG_PARTIDO'])
            dicionario_partido[row['SG_PARTIDO']] = row['QT_VOTOS_NOMINAIS_VALIDOS']

    return dicionario_partido

In [24]:
cidade = 'SÃO PAULO'

cargo = 'Vereador'

dataset_partidos_filtrados_cidade = df_cidade_filtrado_cargo[df_cidade_filtrado_cargo['NM_MUNICIPIO'] == cidade]

dataset_partidos_filtrados_cargo = df_cidade_filtrado_cargo[df_cidade_filtrado_cargo['DS_CARGO'] == cargo]

resultados_partidarios = calculadora_partido(dataset_partidos_filtrados_cargo)

resultados_partidarios

{'PL': 158868,
 'DEM': 429352,
 'PSTU': 2701,
 'PSL': 119303,
 'PRTB': 59285,
 'PP': 112278,
 'REDE': 48530,
 'PC do B': 66804,
 'PSC': 78231,
 'PCO': 300,
 'PODE': 263012,
 'PV': 110978,
 'MDB': 248017,
 'SOLIDARIEDADE': 75650,
 'PCB': 2624,
 'CIDADANIA': 62966,
 'UP': 5067,
 'AVANTE': 57000,
 'PROS': 20943,
 'REPUBLICANOS': 278109,
 'PSDB': 422350,
 'PDT': 46771,
 'PSD': 245992,
 'PATRIOTA': 181283,
 'DC': 9136,
 'PTB': 71057,
 'PSOL': 361634,
 'PMB': 11263,
 'NOVO': 155634,
 'PTC': 17642,
 'PMN': 12268,
 'PT': 562071,
 'PSB': 130063}

In [25]:
df_total_votos_partido = pd.DataFrame(list(resultados_partidarios.items()), columns=['partidos', 'votos_partido'])

df_total_votos_partido.head()

,partidos,votos_partido
0,PL,158868
1,DEM,429352
2,PSTU,2701
3,PSL,119303
4,PRTB,59285


In [34]:
def cria_quociente_partidario(celula):
    
    return int(celula / 80494.21818181819)

In [35]:
df_total_votos_partido['quociente_partidario'] = df_total_votos_partido['votos_partido'].apply(cria_quociente_partidario)

In [36]:
df_total_votos_partido['quociente_partidario'].value_counts()

0    18
1     6
3     4
5     2
2     1
4     1
6     1
Name: quociente_partidario, dtype: int64

In [37]:
df_total_votos_partido.to_csv('quociente_partidario_sp.csv', index=False)

### Candidato mais votados


In [38]:
def calculadora_candidato(dataset_tratado_cidade_cargo):
    dicionario_candidato = {}

    # Conjunto para armazenar nomes de candidatos já encontrados
    partidos_encontrados = set()

    # Iterar pelas linhas do DataFrame
    for index, row in dataset_tratado_cidade_cargo.iterrows():
        # Verificar se o nome do candidato já apareceu
        if row['NM_CANDIDATO'] in partidos_encontrados:
            dicionario_candidato[row['NM_CANDIDATO']] += row['QT_VOTOS_NOMINAIS_VALIDOS']
        else:
            partidos_encontrados.add(row['NM_CANDIDATO'])
            dicionario_candidato[row['NM_CANDIDATO']] = row['QT_VOTOS_NOMINAIS_VALIDOS']

    return dicionario_candidato

In [39]:
dataset_partidos_filtrados_cidade = df_cidade_filtrado_cargo[df_cidade_filtrado_cargo['NM_MUNICIPIO'] == cidade]

dataset_partidos_filtrados_cargo = df_cidade_filtrado_cargo[df_cidade_filtrado_cargo['DS_CARGO'] == cargo]

resultados_candidatos = calculadora_candidato(dataset_partidos_filtrados_cargo)

In [40]:
resultados_candidatos

{'JOSÉ AIRTON FERREIRA DAS NEVES': 6,
 'RODRIGO SCARDELATO GONZALES': 2,
 'MATHEUS DE OLIVEIRA PINTO': 113,
 'ANA PAULA DE SABOIA QUEIROZ': 4,
 'CARLOS ANDRÉ DA SILVA': 0,
 'CÁSSIO LUIZ DE OLIVEIRA': 0,
 'LEILIANE RAFAEL DA SILVA': 22,
 'ADILSON PEREIRA DOS SANTOS': 14,
 'LEANDRO CUSSOLINI FAUSTINO': 8,
 'GRAÇA ERNESTINA ANAY SILVA DIAS': 0,
 'EDNELSON PACHECO FERREIRA': 0,
 'MARIA BEZERRA DE MENEZES SCORZA': 1,
 'CÉSAR BORGES TAVARES': 2,
 'ADRIANO SOARES JUSTINO': 0,
 'BENONI NASCIMENTO DE MOURA': 0,
 'ADAIL LEITE DA SILVA': 1,
 'REINALDO FERNANDES DA SILVA': 2,
 'MARIA DO ROSÁRIO DE FÁTIMA MORAIS': 0,
 'IVETA DE SOUZA ALVES': 0,
 'EVERTON FERREIRA DE SOUSA SANTOS': 8,
 'CLEOMENES ANTUNES JÚNIOR': 3,
 'EDILEUZA GOMES DO NASCIMENTO': 0,
 'ELIANE KANAYAMA': 3,
 'FABIAN ALVES RAMOS': 0,
 'JEAN FERREIRA CAMPOS': 0,
 'LEONARDO FERREIRA LEMOS': 0,
 'LUCIANA MARIA DA SILVA': 0,
 'MARILSON FÉLIX PEREIRA': 7,
 'MOISÉS DE SANTANA': 1,
 'LAURO LISBOA FILHO': 2,
 'MAYKON ALEXANDRE BARROS': 0,
 '

In [41]:
df_total_votos_candidato = pd.DataFrame(list(resultados_candidatos.items()), columns=['candidato', 'votos_candidato'])

In [43]:
def status_candidato(dataset_tratado_cidade_cargo):
    dicionario_candidato = {}

    # Conjunto para armazenar nomes de candidatos já encontrados
    partidos_encontrados = set()

    # Iterar pelas linhas do DataFrame
    for index, row in dataset_tratado_cidade_cargo.iterrows():
        # Verificar se o nome do candidato já apareceu
        if row['NM_CANDIDATO'] in partidos_encontrados:
            pass
        else:
            partidos_encontrados.add(row['NM_CANDIDATO'])
            dicionario_candidato[row['NM_CANDIDATO']] = row['DS_SIT_TOT_TURNO']

    return dicionario_candidato

In [44]:
status = status_candidato(df_cidade_filtrado_cargo)


df_total_votos_candidato['resultado'] = df_total_votos_candidato['candidato'].apply()

TypeError: unhashable type: 'Series'

In [42]:
df_total_votos_candidato.head()

,candidato,votos_candidato
0,JOSÉ AIRTON FERREIRA DAS NEVES,6
1,RODRIGO SCARDELATO GONZALES,2
2,MATHEUS DE OLIVEIRA PINTO,113
3,ANA PAULA DE SABOIA QUEIROZ,4
4,CARLOS ANDRÉ DA SILVA,0


In [45]:
df_total_votos_candidato.to_csv('total_votos_candidato.csv', index=False)